In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import fashion_mnist
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [6]:
from sklearn.model_selection import train_test_split
X_train_clf, X_val_clf, y_train_clf, y_val_clf = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [7]:
y_train_clf_encoded = tf.one_hot(y_train_clf, depth = 10)
y_val_clf_encoded = tf.one_hot(y_val_clf, depth = 10)
y_test_clf_encoded = tf.one_hot(y_test, depth = 10)

In [8]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# data_augmentation = tf.keras.Sequential(
#     [
#         tf.keras.layers.RandomFlip("vertical"),
#         tf.keras.layers.RandomRotation(0.1),
#         tf.keras.layers.RandomZoom(0.1),
#     ]
# )

In [9]:
inputs = tf.keras.Input(shape=(28, 28, 1))
# x = data_augmentation(inputs)
x = inputs

x = tf.keras.layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [16, 32, 64, 128, 256, 512, 1024, 2048]:
    residual = x

    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("tanh")(x)
    x = tf.keras.layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("tanh")(x)
    x = tf.keras.layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("tanh")(x)
    x = tf.keras.layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("tanh")(x)
    x = tf.keras.layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)


    residual = tf.keras.layers.Conv2D(
        size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = tf.keras.layers.add([x, residual])

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
clf1 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [8]:
# clf1.summary()

In [10]:
callbacks_list = [
    EarlyStopping(
        monitor="val_accuracy",
        patience=5,
    ),
    ModelCheckpoint(
        filepath="/content/drive/Shareddrives/ECEN_758_Project/xception.h5",
        monitor="val_accuracy",
        save_best_only=True,
    )
]

In [12]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None
)

In [13]:
clf1.compile(loss="CategoricalCrossentropy",
              optimizer=optimizer,
              metrics=["accuracy", "Precision", "Recall"])
history_clf1 = clf1.fit(
    X_train_clf,
    y_train_clf_encoded,
    epochs=50,
    callbacks=callbacks_list,
    validation_data=(X_val_clf, y_val_clf_encoded)
)

Epoch 1/50
1500/1500 [==============================] - ETA: 0s - loss: 0.5896 - accuracy: 0.7963 - precision: 0.8423 - recall: 0.7562

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1500/1500 [==============================] - 110s 51ms/step - loss: 0.5896 - accuracy: 0.7963 - precision: 0.8423 - recall: 0.7562 - val_loss: 0.4881 - val_accuracy: 0.8414 - val_precision: 0.8608 - val_recall: 0.8237
Epoch 2/50
1500/1500 [==============================] - 63s 42ms/step - loss: 0.4209 - accuracy: 0.8559 - precision: 0.8829 - recall: 0.8305 - val_loss: 0.4165 - val_accuracy: 0.8515 - val_precision: 0.8850 - val_recall: 0.8202
Epoch 3/50
1500/1500 [==============================] - 65s 43ms/step - loss: 0.3745 - accuracy: 0.8733 - precision: 0.8955 - recall: 0.8522 - val_loss: 0.4051 - val_accuracy: 0.8648 - val_precision: 0.8851 - val_recall: 0.8469
Epoch 4/50
1500/1500 [==============================] - 65s 43ms/step - loss: 0.3444 - accuracy: 0.8826 - precision: 0.9020 - recall: 0.8649 - val_loss: 0.3534 - val_accuracy: 0.8815 - val_precision: 0.9065 - val_recall: 0.8610
Epoch 5/50
1500/1500 [==============================] - 59s 40ms/step - loss: 0.3198 - accuracy: 0

In [2]:
xception = tf.keras.saving.load_model('/content/drive/Shareddrives/ECEN_758_Project/xception_stable.h5')

In [13]:
test_loss, test_acc, test_precision, test_recall = xception.evaluate(x_test, y_test_clf_encoded)
your_score = round(test_acc*1000) / 10
print(f"Your Score: {your_score}")

313/313 [==============================] - 4s 13ms/step - loss: 0.2964 - accuracy: 0.9042 - precision: 0.9135 - recall: 0.8994
Your Score: 90.4
